# Analiza i preprocessing danych

Organizator: Koło naukowe BioMedicalAI  
![biomedical.svg](biomedical.svg)

## Analiza danych (20m)


In [201]:
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.datasets

# 1) Wstępna analiza 
# Opis datasetu (https://scikit-learn.org/1.5/datasets/toy_dataset.html#wine-recognition-dataset)
# NOTE: wiedza dziedzinowa - 
dataset  = sklearn.datasets.load_wine(as_frame=True)["frame"]

In [ ]:
# Ilość próbek
len(dataset)

In [ ]:
# Kolumny, typy danych i ilość pustych wartości
dataset.info()

In [ ]:
# Przykładowe dane z przodu
dataset.head(n=10)

In [ ]:
# Przykładowe dane z końca datasetu i porównanie czy dane różnią się mocno pomiędzy przodem a końcem zbioru danych. 
dataset.tail(n=10)

In [ ]:
# Statystyki danych numeryczych
dataset.describe().apply(lambda x: x.apply('{0:.5f}'.format))

In [ ]:
# Wizualizacja poszczególnych kolumn na wykresach
for column in dataset.columns:
    sns.boxplot(dataset, y=column)
    plt.show()

In [ ]:
# Badanie zbalansowania danych
sns.histplot(dataset, x="target")

In [ ]:
# Wizualizacja poszczególnych kolumn na wykresach
for column in dataset.drop("target", axis=1).columns:
    sns.kdeplot(dataset, x=column, hue="target", palette="tab10")
    plt.show()

In [ ]:
# Pairplot
sns.pairplot(dataset, hue='target', palette="tab10")

In [ ]:
# Heatmapa korelacji między kolumnami
plt.figure(figsize = (10,10))
sns.heatmap(dataset.drop("target", axis=1).corr(), annot = True)

## Preprocessing (30m)
Preprocessing ma na celu przystosowanie danych pod dalsze ich użycie przez człowieka lub model.

In [212]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import sklearn.metrics
import numpy as np


In [ ]:
# Podział kolumn na kolumny wejściowe i wyjściowe
y = dataset["target"].to_numpy()
X = dataset.drop("target", axis=1).to_numpy()
X.shape, y.shape

In [214]:
# Podział na dane treningowe, walidacyjne i testowe
# Dane treningowe - na tych danych będziemy trenować model
# Dane walidacyjne - na tych danych będziemy dopasowywać model (hiperparementry modelu etc)
# Dane testowe - na tych danych będziemy testować ostateczną jakośc modelu
# Zazwyczaj dane dzieli się w stosunku 70%:10%:20%
# Innym sposobem może być zastosowanie cross-validacji w celu redukcji wpływu losowego podziału na ostateczne wyniki działania modelu
# https://medium.com/@masadeghi6/how-to-split-your-data-for-machine-learning-eae893a8799c
seed = 21

X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=0.25, random_state=seed) 

### Zadania a metryki
| Zadanie | Funkcja straty | Metryki |
|-|-|-|
| Regresja | MSE, MAE | R2, MSE, MAE, RMSE |
| Klasyfikacja | CE, KL Divergence | Dokładność(Accuracy), Precyzja(Precision), Czułość(Recall), F1, ROC_AUC, MCC |

Accuracy - jaki procent przypadków raka oraz braku raka poprawnie wykryliśmy

$Accuracy = \frac{TP+TN}{TP+TN+FP+FN}$


Precision - jaki procent wykrytych przypadków raka jest faktycznie rakiem

$Precision = \frac{TP}{TP+FP}$

F1 - średnia harmoniczna precision i recall

$F1 = \frac{2*Precision*Recall}{Precision+Recall} = \frac{2*TP}{2*TP+FP+FN}$

ROC_AUC - jak zmienia się działanie klasyfikatora w zależności od przyjętego progu klasyfikacji

Matthews’s Correlation Coefficient (MCC) - metryka jakości klasyfikatora odporniejsza na niezbalansowane datasety

$MCC=TP×TN−FP×FN(TP+FP)(TP+FN)(TN+FP)(TN+FN)$

In [ ]:
# Krok 1. Bez przetwarzania danych wejściowych
# Ważnym elementem jest poprawność metryk wykorzystywanych w procesie oceniania jakości modelu.
# W tym celu warto wykorzystać metryki dostarczane przez biblioteki.

seed = 21
best_model = None
best_size = None 
best_score = 0
for size in [(100,), (100, 50), (50, 50, 10), (100, 100, 100, 100)]:
    model = MLPClassifier(hidden_layer_sizes=size, random_state=seed, max_iter=2000)
    model.fit(X_train, y_train)
    val_score = sklearn.metrics.accuracy_score(y_val, model.predict(X_val))
    test_score = sklearn.metrics.accuracy_score(y_test, model.predict(X_test))
    print("VAL + TEST score", size, round(val_score), round(test_score))
    if val_score > best_score:
        best_score = val_score
        best_model = model
        best_size = size

# Model wybrany na podstawie danych walidacyjych wcale nie musi być najlepszy (!)
print("TEST score based on VAL score", best_size, round(sklearn.metrics.accuracy_score(y_test, best_model.predict(X_test))))
    

In [ ]:
np.vstack([y_test, best_model.predict(X_test)]).T

In [ ]:
# Raport z klasyfikacji
print(sklearn.metrics.classification_report(y_test, best_model.predict(X_test), zero_division=np.nan))

### Micro a macro metryka

$Precision = \frac{TP}{TP+FP}$

$Precision_{micro} = \frac{(TP_0+TP_1+TP_2)}{(TP_0+TP_1+TP_2)+(FP_0+FP_1+FP_2)}$

$Precision_{macro} = \frac{Precision_0 + Precision_1 + Precision_2}{3}$

Makro uśrednianie uśrednia pomiędzy klasami, micro uśrednianie uśrednia pomiędzy elementami

In [ ]:
# Macierz pomyłek
sklearn.metrics.confusion_matrix(y_test, best_model.predict(X_test))

In [ ]:
# Macierz pomyłek
# NOTE: Określanie błędów modelu i środki zaradcze
sns.heatmap(sklearn.metrics.confusion_matrix(y_test, best_model.predict(X_test)), annot=True, cmap="Blues")

In [ ]:
# Preprocessing - scalers https://scikit-learn.org/dev/auto_examples/preprocessing/plot_all_scaling.html#original-data
# NOTE: Data leakage

seed = 21
best_model = None
best_scaler = None 
best_score = 0
sc = sklearn.preprocessing.StandardScaler()


for scaler in [
    sklearn.preprocessing.StandardScaler,
    sklearn.preprocessing.MinMaxScaler,
    sklearn.preprocessing.RobustScaler,
    sklearn.preprocessing.PowerTransformer,
    sklearn.preprocessing.Normalizer,
]:
    sc = scaler()
    X_train_transformed = sc.fit_transform(X_train)
    X_val_transformed = sc.transform(X_val)
    X_test_transformed = sc.transform(X_test)

    model = MLPClassifier(hidden_layer_sizes=(100, 50), random_state=seed, max_iter=2000)
    model.fit(X_train_transformed, y_train)
    val_score = sklearn.metrics.accuracy_score(y_val, model.predict(X_val_transformed))
    test_score = sklearn.metrics.accuracy_score(y_test, model.predict(X_test_transformed))
    print("VAL + TEST score", scaler, round(val_score, 2), round(test_score, 2))
    if val_score > best_score:
        best_score = val_score
        best_model = model
        best_scaler = scaler


bsc = best_scaler()
X_train_transformed = bsc.fit_transform(X_train)
X_val_transformed = bsc.transform(X_val)
X_test_transformed = bsc.transform(X_test)
print("TEST score based on VAL score", best_scaler, round(sklearn.metrics.accuracy_score(y_test, best_model.predict(X_test_transformed))))
print(sklearn.metrics.classification_report(y_test, best_model.predict(X_test_transformed), zero_division=np.nan))
print(round(sklearn.metrics.matthews_corrcoef(y_test, best_model.predict(X_test_transformed))))
sns.heatmap(sklearn.metrics.confusion_matrix(y_test, best_model.predict(X_test_transformed)), annot=True, cmap="Blues")

In [ ]:
# Dyskretyzacja danych

seed = 21
sc = sklearn.preprocessing.KBinsDiscretizer(n_bins=5)
X_train_transformed = sc.fit_transform(X_train)
X_val_transformed = sc.transform(X_val)
X_test_transformed = sc.transform(X_test)

model = MLPClassifier(hidden_layer_sizes=(100, 50), random_state=seed, max_iter=2000)
model.fit(X_train_transformed, y_train)
val_score = sklearn.metrics.accuracy_score(y_val, model.predict(X_val_transformed))
test_score = sklearn.metrics.accuracy_score(y_test, model.predict(X_test_transformed))
print("VAL + TEST score", round(val_score), round(test_score))

print(sklearn.metrics.classification_report(y_test, model.predict(X_test_transformed), zero_division=np.nan))
sns.heatmap(sklearn.metrics.confusion_matrix(y_test, model.predict(X_test_transformed)), annot=True, cmap="Blues")
plt.show()

In [ ]:
# OneHotEncoding w celu uproszenia klasyfikacji modelom NN
one_hot_y_train = sklearn.preprocessing.OneHotEncoder().fit_transform(y_train.reshape(-1, 1))
one_hot_y_train.toarray()

In [ ]:
# ROC AUC dla gorszego modelu
model = MLPClassifier(hidden_layer_sizes=(100, 50, 10), random_state=seed, max_iter=2000)
model.fit(X_train, y_train)
label_binarizer = sklearn.preprocessing.OneHotEncoder().fit(y_train.reshape(-1, 1))
y_onehot_test = label_binarizer.transform(y_test.reshape(-1, 1)).toarray()
y_score = model.predict_proba(X_test)

display = sklearn.metrics.RocCurveDisplay.from_predictions(
    y_onehot_test.ravel(),
    y_score.ravel(),
    name="micro-average OvR",
    color="darkorange",
    plot_chance_level=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="Micro-averaged One-vs-Rest\nReceiver Operating Characteristic",
)

In [ ]:
# Cross-validation ze stratyfikacją po klasie
scores = []

external_skf = sklearn.model_selection.StratifiedKFold(n_splits=5)
for tv, test in external_skf.split(X, y):
    internal_skf = sklearn.model_selection.StratifiedKFold(n_splits=5)
    for train, val in internal_skf.split(X[tv], y[tv]):
        seed = 21
        sc = sklearn.preprocessing.StandardScaler()
        X_train_transformed = sc.fit_transform(X[train])
        X_val_transformed = sc.transform(X[val])
        X_test_transformed = sc.transform(X[test])

        model = MLPClassifier(hidden_layer_sizes=(100, 50), random_state=seed, max_iter=2000)
        model.fit(X_train_transformed, y[train])
        val_score = sklearn.metrics.matthews_corrcoef(y[val], model.predict(X_val_transformed))
        test_score = sklearn.metrics.matthews_corrcoef(y[test], model.predict(X_test_transformed))
        scores.append(test_score)
        print("VAL score", round(val_score, 2))

np_scores = np.array(scores)
print(np_scores.min().round(2), np_scores.mean().round(2), np_scores.max().round(2))


## Ankieta
!["Ankieta"](./ankieta.png)  

## Backpropagation step by step

In [ ]:
x = np.array([[1, 2]])

w_1 = np.array([[1.0, 2.0], [3.0, 4.0]])
b_1 = np.array([[5.0, 6.0]])
w_2 = np.array([[7.0], [8.0]])
b_2 = np.array([[9.0]])

weights = np.array([[4], [-1]])
bias = np.array([0.5])
lr = 0.001
y_true = x @ weights + bias

for i in range(100):
    y_1 = (x @ w_1 + b_1)
    y_2 = (y_1 @ w_2 + b_2)
    y_pred = y_2

    error = 2. * (y_pred - y_true) / len(x)
    e_3 = error

    wg_2 = y_1.T @ e_3
    wb_2 = np.sum(e_3, axis=0)
    e_1 = e_3 @ w_2.T
    w_2 -= wg_2 * lr
    b_2 -= wb_2 * lr


    wg_1 = x.T @ e_1
    wb_1 = np.sum(e_1, axis=0)
    e_0 = e_1 @ w_1.T
    w_1 -= wg_1 * lr
    b_1 -= wb_1 * lr



    print(y_1, y_2)
    print("W2")
    print(e_3)
    print(wg_2)
    print(wb_2)
    print("W1")
    print(e_1)
    print(wg_1)
    print(wb_1)
    print()